# Tech Intro

## Main tables

In [1]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [2]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = real_estate_list[4] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [3]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

22/11/06 23:21:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/06 23:21:20 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/11/06 23:21:20 WARN Utils: Service 'SparkUI' could not bind on port 4080. Attempting port 4081.
22/11/06 23:21:20 WARN Utils: Service 'SparkUI' could not bind on port 4081. Attempting port 4082.
22/11/06 23:21:20 WARN Utils: Service 'SparkUI' could not bind on port 4082. Attempting port 4083.
22/11/06 23:21:20 WARN Utils: Service 'SparkUI' could not bind on port 4083. Attempting port 4084.
22/11/06 23:21:21 WARN FairSchedulableBuilder: Fair Sc

## Read the spec-file from Hadoop and show some data sample

In [4]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+-------+------------+-----------------+-------+-----+----+----------+----------+
|id     |ag_object_id|ag_ad_platform_id|publish|views|url |created_at|updated_at|
+-------+------------+-----------------+-------+-----+----+----------+----------+
|1000005|102260      |844              |0      |0    |null|1621409479|1621409479|
|1000006|102260      |845              |0      |0    |null|1621409479|1621409479|
|1000008|102260      |848              |0      |0    |null|1621409479|1621409479|
|1000019|102261      |276              |0      |0    |null|1621409531|1621409531|
|100002 |17786       |29               |0      |0    |null|0         |0         |
|1000020|102261      |277              |0      |0    |null|1621409531|1621409531|
|1000022|102261      |646              |0      |0    |null|1621409531|1621409531|
|1000030|102261      |846              |0      |0    |null|1621409531|1621409531|
|1000035|102261      |852              |0      |0    |null|1621409531|1621409531|
|1000036|102261 

## Show columns' properties

In [5]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- ag_object_id: string (nullable = true)
 |-- ag_ad_platform_id: string (nullable = true)
 |-- publish: string (nullable = true)
 |-- views: string (nullable = true)
 |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- updated_at: string (nullable = true)



## Total rows (count)

In [6]:
eda_df.count()

4637909

## Simple func to execute and show the result of the SQL-instruction

In [7]:
def execute(select_instruction: str = "select count(*) from realEstate") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from realEstate"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show()

## Create the temp-table-view from the DF (called as "eda_sql")

In [8]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [9]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
| 4637909|
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [10]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,ag_object_id,ag_ad_platform_id,publish,views,url,created_at,updated_at
0,0,0,0,0,0,4396211,0,0


In [11]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,ag_object_id,ag_ad_platform_id,publish,views,url,created_at,updated_at
0,0,0,0,2108773,4637909,0,397416,324740


In [12]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,ag_object_id,ag_ad_platform_id,publish,views,url,created_at,updated_at
0,0,0,0,0,0,6985,0,0


In [13]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
ag_object_id,0,0,0
ag_ad_platform_id,0,0,0
publish,0,2108773,0
views,0,4637909,0
url,4396211,0,6985
created_at,0,397416,0
updated_at,0,324740,0


In [14]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [15]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
ag_object_id,0.000,0.000,0.000
ag_ad_platform_id,0.000,0.000,0.000
publish,0.000,45.468,0.000
views,0.000,100.000,0.000
url,94.789,0.000,0.151
created_at,0.000,8.569,0.000
updated_at,0.000,7.002,0.000


### SQL-instructions

In [21]:
#Find dupes or misunderstanding or useless static values 4 637 909 total
execute("SELECT id, created_at, updated_at\
        FROM eda_sql\
        WHERE updated_at = 0 OR created_at = 0")


for col in eda_df.columns: 
    execute("select count(distinct("+ col + ")) from eda_sql")


+------+----------+----------+
|    id|created_at|updated_at|
+------+----------+----------+
|100002|         0|         0|
|100028|         0|         0|
|100066|         0|         0|
|100077|         0|1627365437|
|100124|         0|         0|
|100137|         0|         0|
|100144|         0|         0|
|100153|         0|         0|
|100163|         0|         0|
|100183|         0|         0|
|100220|         0|         0|
|100227|         0|         0|
|100241|         0|         0|
|100245|         0|         0|
|100263|         0|         0|
|100267|         0|         0|
|100278|         0|         0|
|100290|         0|         0|
|100340|         0|         0|
|100390|         0|1620465020|
+------+----------+----------+
only showing top 20 rows



+------------------+
|count(DISTINCT id)|
+------------------+
|           4637909|
+------------------+



+----------------------------+
|count(DISTINCT ag_object_id)|
+----------------------------+
|                      298630|
+----------------------------+

+---------------------------------+
|count(DISTINCT ag_ad_platform_id)|
+---------------------------------+
|                             2228|
+---------------------------------+

+-----------------------+
|count(DISTINCT publish)|
+-----------------------+
|                      2|
+-----------------------+

+---------------------+
|count(DISTINCT views)|
+---------------------+
|                    1|
+---------------------+

+-------------------+
|count(DISTINCT url)|
+-------------------+
|             233696|
+-------------------+



+--------------------------+
|count(DISTINCT created_at)|
+--------------------------+
|                    398021|
+--------------------------+



+--------------------------+
|count(DISTINCT updated_at)|
+--------------------------+
|                    747178|
+--------------------------+



## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [22]:
spark.stop()

I1106 23:46:04.647354 37901 sched.cpp:2174] Asked to stop the driver
I1106 23:46:04.647500 38132 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-53207
